In [1]:
import pickle

In [2]:
with open('/project_final/orbitals.pkl', 'rb') as f:
    orbitals = pickle.load(f)

with open('/project_final/maneuvers.pkl', 'rb') as f:
    maneuvers = pickle.load(f)

In [3]:
man_s3a = maneuvers['s3a']
man_fy2d = maneuvers['FY2D']
man_saral = maneuvers['srl']

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
baseline_s3a_df = pd.read_csv("/project_final/result/s3a_baseline_residuals.csv", parse_dates=["date"])
arima_s3a_df = pd.read_csv("/project_final/result/s3a_arima_residuals.csv", parse_dates=["date"])
xgb_s3a_df = pd.read_csv("/project_final/result/s3a_xgb_residuals.csv", parse_dates=["day_time"])
xgb_diff_s3a_df = pd.read_csv("/project_final/result/s3a_xgb_residuals_diff.csv", parse_dates=["day_time"])
lstm_s3a_df = pd.read_csv("/project_final/result/s3a_lstm_residuals.csv", parse_dates=["day_time"])
lstm_ae_s3a_df = pd.read_csv("/project_final/result/s3a_lstm_autoencoder_residuals.csv", parse_dates=["timestamp"])

baseline_fy2d_df = pd.read_csv("/project_final/result/fy2d_baseline_residuals.csv", parse_dates=["date"])
arima_fy2d_df = pd.read_csv("/project_final/result/fy2d_arima_residuals.csv", parse_dates=["date"])
xgb_fy2d_df = pd.read_csv("/project_final/result/fy2d_xgb_residuals.csv", parse_dates=["day_time"])
xgb_diff_fy2d_df = pd.read_csv("/project_final/result/fy2d_xgb_residuals_diff.csv", parse_dates=["day_time"])
lstm_fy2d_df = pd.read_csv("/project_final/result/fy2d_lstm_residuals.csv", parse_dates=["day_time"])
lstm_ae_fy2d_df = pd.read_csv("/project_final/result/fy2d_lstm_autoencoder_residuals.csv", parse_dates=["timestamp"])

baseline_saral_df = pd.read_csv("/project_final/result/saral_baseline_residuals.csv", parse_dates=["date"])
arima_saral_df = pd.read_csv("/project_final/result/saral_arima_residuals.csv", parse_dates=["date"])
xgb_saral_df = pd.read_csv("/project_final/result/saral_xgb_residuals.csv", parse_dates=["day_time"])
xgb_diff_saral_df = pd.read_csv("/project_final/result/saral_xgb_residuals_diff.csv", parse_dates=["day_time"])
lstm_saral_df = pd.read_csv("/project_final/result/saral_lstm_residuals.csv", parse_dates=["day_time"])
lstm_ae_saral_df = pd.read_csv("/project_final/result/saral_lstm_autoencoder_residuals.csv", parse_dates=["timestamp"])


In [6]:
xgb_s3a_df.rename(columns={"day_time": "date"}, inplace=True)
xgb_fy2d_df.rename(columns={"day_time": "date"}, inplace=True)
xgb_saral_df.rename(columns={"day_time": "date"}, inplace=True)
xgb_diff_s3a_df.rename(columns={"day_time": "date"}, inplace=True)
xgb_diff_fy2d_df.rename(columns={"day_time": "date"}, inplace=True)
xgb_diff_saral_df.rename(columns={"day_time": "date"}, inplace=True)
lstm_s3a_df.rename(columns={"day_time": "date"}, inplace=True)
lstm_fy2d_df.rename(columns={"day_time": "date"}, inplace=True)
lstm_saral_df.rename(columns={"day_time": "date"}, inplace=True)
lstm_ae_s3a_df.rename(columns={"timestamp": "date"}, inplace=True)
lstm_ae_fy2d_df.rename(columns={"timestamp": "date"}, inplace=True)
lstm_ae_saral_df.rename(columns={"timestamp": "date"}, inplace=True)

In [7]:
baseline_s3a_df.drop(columns=['actual','fitted'],inplace=True)
arima_s3a_df.drop(columns=['actual','fitted'],inplace=True)
xgb_s3a_df.drop(columns=['actual','pred'],inplace=True)
xgb_diff_s3a_df.drop(columns=['actual','pred'],inplace=True)
lstm_s3a_df.drop(columns=['actual', 'pred'],inplace=True)

baseline_fy2d_df.drop(columns=['actual','fitted'],inplace=True)
arima_fy2d_df.drop(columns=['actual','fitted'],inplace=True)
xgb_fy2d_df.drop(columns=['actual','pred'],inplace=True)
xgb_diff_fy2d_df.drop(columns=['actual','pred'],inplace=True)
lstm_fy2d_df.drop(columns=['actual', 'pred'],inplace=True)

baseline_saral_df.drop(columns=['actual','fitted'],inplace=True)
arima_saral_df.drop(columns=['actual','fitted'],inplace=True)
xgb_saral_df.drop(columns=['actual','pred'],inplace=True)
xgb_diff_saral_df.drop(columns=['actual','pred'],inplace=True)
lstm_saral_df.drop(columns=['actual', 'pred'],inplace=True)

In [8]:
def align_to_reference_dates(
    ref_df,
    *other_dfs,
    date_col="date",
    strict=True,
    abs_residual=True
):

    ref = ref_df.copy()
    ref[date_col] = pd.to_datetime(ref[date_col], errors="coerce")
    ref = ref.dropna(subset=[date_col]).sort_values(date_col)

    start, end = ref[date_col].min(), ref[date_col].max()
    ref_dates = pd.DatetimeIndex(ref[date_col].unique()) if strict else None

    aligned_list = []

    def _post_process(df):

        if abs_residual and "residual" in df.columns:
            df["residual"] = df["residual"].abs()
        
        df = df.sort_values(date_col).reset_index(drop=True)
        # df[date_col] = df[date_col].dt.strftime(fmt)
        return df

    for df in other_dfs:
        d = df.copy()
        d[date_col] = pd.to_datetime(d[date_col], errors="coerce")
        d = d.dropna(subset=[date_col])

        if strict:
            d = d[d[date_col].isin(ref_dates)]
        else:
            d = d[(d[date_col] >= start) & (d[date_col] <= end)]

        aligned_list.append(_post_process(d))

    ref_out = ref.copy()
    if abs_residual and "residual" in ref_out.columns:
        ref_out["residual"] = ref_out["residual"].abs()
    ref_out = ref_out.sort_values(date_col).reset_index(drop=True)

    return (ref_out, *aligned_list)

In [9]:
print(baseline_s3a_df.head(1), "\n", arima_s3a_df.head(1), "\n", 
      xgb_s3a_df.head(1), "\n", xgb_diff_s3a_df.head(1), "\n", 
      lstm_s3a_df.head(1), "\n", lstm_ae_s3a_df.head(1))

        date  residual
0 2016-03-04  0.000062 
         date      residual
0 2016-03-04 -7.399500e-08 
         date  residual
0 2016-03-07 -0.003524 
         date  residual
0 2016-03-12 -0.082285 
         date  residual
0 2016-03-09  0.155023 
         date  residual
0 2016-03-08  0.026827


In [10]:
xgb_diff_s3a_aligned, lstm_s3a_aligned, arima_s3a_aligned, baseline_s3a_aligned, xgb_s3a_aligned, lstm_ae_s3a_aligned = align_to_reference_dates(
    xgb_diff_s3a_df, lstm_s3a_df, arima_s3a_df, baseline_s3a_df, xgb_s3a_df, lstm_ae_s3a_df,
    date_col="date", strict=True, abs_residual=True
)

In [11]:
print(baseline_s3a_aligned.head(1), "\n", arima_s3a_aligned.head(1), "\n", 
      xgb_s3a_aligned.head(1), "\n", xgb_diff_s3a_aligned.head(1), "\n", 
      lstm_s3a_aligned.head(1), "\n", lstm_ae_s3a_aligned.head(1))

        date      residual
0 2016-03-12  4.788452e-08 
         date      residual
0 2016-03-12  3.894827e-08 
         date  residual
0 2016-03-12  0.000131 
         date  residual
0 2016-03-12  0.082285 
         date  residual
0 2016-03-12  0.079569 
         date  residual
0 2016-03-12  0.041956


In [12]:
print(baseline_fy2d_df.head(1), "\n", arima_fy2d_df.head(1), "\n", 
      xgb_fy2d_df.head(1), "\n", xgb_diff_fy2d_df.head(1), "\n", 
      lstm_fy2d_df.head(1), "\n", lstm_ae_fy2d_df.head(1))

        date  residual
0 2011-01-27  0.000004 
         date      residual
0 2011-01-27  1.000031e-07 
         date  residual
0 2011-02-02 -0.001062 
         date  residual
0 2011-02-01  0.141243 
         date  residual
0 2011-01-30 -0.072843 
         date  residual
0 2011-01-29  0.010002


In [13]:
xgb_fy2d_aligned, lstm_fy2d_aligned, arima_fy2d_aligned, baseline_fy2d_aligned, xgb_diff_fy2d_aligned, lstm_ae_fy2d_aligned = align_to_reference_dates(
    xgb_fy2d_df, lstm_fy2d_df, arima_fy2d_df, baseline_fy2d_df, xgb_diff_fy2d_df, lstm_ae_fy2d_df,
    date_col="date", strict=True, abs_residual=True
)

In [14]:
print(baseline_fy2d_aligned.head(1), "\n", arima_fy2d_aligned.head(1), "\n", 
      xgb_fy2d_aligned.head(1), "\n", xgb_diff_fy2d_aligned.head(1), "\n", 
      lstm_fy2d_aligned.head(1), "\n", lstm_ae_fy2d_aligned.head(1))

        date      residual
0 2011-02-02  2.561543e-09 
         date      residual
0 2011-02-02  3.042044e-08 
         date  residual
0 2011-02-02  0.001062 
         date  residual
0 2011-02-02  0.041915 
         date  residual
0 2011-02-02   0.03284 
         date  residual
0 2011-02-02  0.030831


In [15]:
print(baseline_saral_df.head(1), "\n", arima_saral_df.head(1), "\n", 
      xgb_saral_df.head(1), "\n", xgb_diff_saral_df.head(1), "\n", 
      lstm_saral_df.head(1), "\n", lstm_ae_saral_df.head(1))

        date  residual
0 2013-03-10  0.000063 
         date      residual
0 2013-03-10  4.357328e-08 
         date  residual
0 2013-03-18  0.004952 
         date  residual
0 2013-03-17  -1.47788 
         date  residual
0 2013-03-14 -0.698613 
         date  residual
0 2013-03-13  0.117855


In [16]:
xgb_saral_aligned, lstm_saral_aligned, arima_saral_aligned, baseline_saral_aligned, xgb_diff_saral_aligned, lstm_ae_saral_aligned = align_to_reference_dates(
    xgb_saral_df, lstm_saral_df, arima_saral_df, baseline_saral_df, xgb_diff_saral_df, lstm_ae_saral_df,
    date_col="date", strict=True, abs_residual=True
)

In [17]:
print(baseline_saral_aligned.head(1), "\n", arima_saral_aligned.head(1), "\n", 
      xgb_saral_aligned.head(1), "\n", xgb_diff_saral_aligned.head(1), "\n", 
      lstm_saral_aligned.head(1), "\n", lstm_ae_saral_aligned.head(1))

        date      residual
0 2013-03-18  8.381405e-08 
         date      residual
0 2013-03-18  1.068347e-07 
         date  residual
0 2013-03-18  0.004952 
         date  residual
0 2013-03-18  0.171789 
         date  residual
0 2013-03-18  0.000246 
         date  residual
0 2013-03-18  0.004757


## man align with res abs

In [18]:
man_dates_s3a = pd.to_datetime(man_s3a["median_day_time"])
man_dates_s3a = man_dates_s3a.sort_values()

man_dates_fy2d = pd.to_datetime(man_fy2d["median_day_time"])
man_dates_fy2d = man_dates_fy2d.sort_values()

man_dates_saral = pd.to_datetime(man_saral["median_day_time"])
man_dates_saral = man_dates_saral.sort_values()

In [19]:
man_dates_s3a = man_dates_s3a[6:-1]
man_dates_fy2d = man_dates_fy2d.copy()
man_dates_saral = man_dates_saral[7:-1]

## threshold

In [20]:
def convert_timestamp_series_to_epoch(series):
    return (
        (series - pd.Timestamp(year=1970, month=1, day=1)) // pd.Timedelta(seconds=1)
    ).values

In [21]:
def compute_simple_matching_precision_recall_for_one_threshold(
    matching_max_days,
    threshold,
    series_ground_truth_manoeuvre_timestamps,
    series_predictions,
):
    matching_max_distance_seconds = pd.Timedelta(days=matching_max_days).total_seconds()
    dict_predictions_to_ground_truth = {}
    dict_ground_truth_to_predictions = {}

    manoeuvre_timestamps_seconds = convert_timestamp_series_to_epoch(series_ground_truth_manoeuvre_timestamps)
    pred_time_stamps_seconds = convert_timestamp_series_to_epoch(series_predictions.index)
    predictions = series_predictions.to_numpy()

    for i in range(predictions.shape[0]):
        if predictions[i] >= threshold:
            left_index = np.searchsorted(
                manoeuvre_timestamps_seconds, pred_time_stamps_seconds[i]
            )

            if left_index != 0:
                left_index -= 1

            index_of_closest = left_index

            if (left_index < len(series_ground_truth_manoeuvre_timestamps) - 1) and (
                abs(manoeuvre_timestamps_seconds[left_index] - pred_time_stamps_seconds[i])
                > abs(manoeuvre_timestamps_seconds[left_index + 1] - pred_time_stamps_seconds[i])
            ):
                index_of_closest = left_index + 1

            diff = abs(manoeuvre_timestamps_seconds[index_of_closest] - pred_time_stamps_seconds[i])

            if diff < matching_max_distance_seconds:
                dict_predictions_to_ground_truth[i] = (index_of_closest, diff)
                if index_of_closest in dict_ground_truth_to_predictions:
                    dict_ground_truth_to_predictions[index_of_closest].append(i)
                else:
                    dict_ground_truth_to_predictions[index_of_closest] = [i]

    positive_prediction_indices = np.argwhere(predictions >= threshold)[:, 0]
    list_false_positives = [
        pred_ind for pred_ind in positive_prediction_indices if pred_ind not in dict_predictions_to_ground_truth
    ]
    list_false_negatives = [
        true_ind for true_ind in np.arange(0, len(series_ground_truth_manoeuvre_timestamps))
        if true_ind not in dict_ground_truth_to_predictions
    ]

    precision = len(dict_ground_truth_to_predictions) / (len(dict_ground_truth_to_predictions) + len(list_false_positives)) if (len(dict_ground_truth_to_predictions) + len(list_false_positives)) > 0 else 0.0
    recall = len(dict_ground_truth_to_predictions) / (len(dict_ground_truth_to_predictions) + len(list_false_negatives)) if (len(dict_ground_truth_to_predictions) + len(list_false_negatives)) > 0 else 0.0

    return (precision, recall)

In [22]:
def find_best_f1(thresholds, precision_list, recall_list):
    f1_scores = [2 * (p * r) / (p + r) if (p + r) > 0 else 0.0 for p, r in zip(precision_list, recall_list)]
    max_index = int(np.argmax(f1_scores))
    return (
        thresholds[max_index],      
        f1_scores[max_index],       
        precision_list[max_index],  
        recall_list[max_index]     
    )

In [23]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [24]:
def _safe_series_datetime_index(series: pd.Series) -> pd.Series:
    s = series.copy()
    if not isinstance(s.index, pd.DatetimeIndex):
        s.index = pd.to_datetime(s.index)
    return s.sort_index()

def _clean_thresholds(arr_like) -> np.ndarray:
    arr = np.asarray(arr_like, dtype=float).ravel()
    arr = arr[np.isfinite(arr)]
    return np.unique(np.sort(arr))

def _nan_to_zero(x):
    if x is None:
        return 0.0
    try:
        return 0.0 if np.isnan(x) else float(x)
    except TypeError:
        return float(x)

def pr_curves_for_models_across_windows(
    model_series: dict,                       
    man_dates: pd.DatetimeIndex,              
    window_days=(1, 3, 5),                    
    thresholds_by_model: dict | None = None,  
    return_dataframe: bool = False            
):

    results = defaultdict(dict)

    # 
    man_dates = pd.to_datetime(pd.Index(man_dates)).sort_values().unique()

    # 
    df_rows = []

    for model_name, series in model_series.items():
        s = _safe_series_datetime_index(series)

        # 
        if thresholds_by_model and model_name in thresholds_by_model:
            thr_arr = _clean_thresholds(thresholds_by_model[model_name])
        else:
            thr_arr = _clean_thresholds(s.values)

        # 
        if thr_arr.size == 0:
            for w in window_days:
                results[model_name][f"{w}d"] = {
                    "thresholds": np.array([]),
                    "precision": [],
                    "recall": [],
                }
            continue

        # 
        for w in window_days:
            p_list, r_list = [], []
            for thr in thr_arr:
                p, r = compute_simple_matching_precision_recall_for_one_threshold(
                    matching_max_days=float(w),
                    threshold=float(thr),
                    series_ground_truth_manoeuvre_timestamps=man_dates,
                    series_predictions=s
                )
                p = _nan_to_zero(p)
                r = _nan_to_zero(r)
                p_list.append(p)
                r_list.append(r)

                if return_dataframe:
                    df_rows.append(
                        (model_name, int(w), float(thr), float(p), float(r))
                    )

            results[model_name][f"{w}d"] = {
                "thresholds": thr_arr,
                "precision": p_list,
                "recall": r_list,
            }

    if return_dataframe:
        df_long = pd.DataFrame(
            df_rows, columns=["model", "window_days", "threshold", "precision", "recall"]
        ).sort_values(["model", "window_days", "threshold"]).reset_index(drop=True)
        return results, df_long

    return results

In [25]:
model_series_s3a = {
    "baseline": baseline_s3a_aligned.set_index("date")["residual"],
    "arima":    arima_s3a_aligned.set_index("date")["residual"],
    "xgb": xgb_s3a_aligned.set_index("date")["residual"],
    "xgb_diff": xgb_diff_s3a_aligned.set_index("date")["residual"],
    "lstm":     lstm_s3a_aligned.set_index("date")["residual"],
    "lstm_ae":     lstm_ae_s3a_aligned.set_index("date")["residual"],
}
res_s3a, df_pr_s3a = pr_curves_for_models_across_windows(
    model_series=model_series_s3a,
    man_dates=man_dates_s3a,
    window_days=(1,3,5),
    return_dataframe=True
)

In [26]:
model_series_fy2d = {
    "baseline": baseline_fy2d_aligned.set_index("date")["residual"],
    "arima":    arima_fy2d_aligned.set_index("date")["residual"],
    "xgb": xgb_fy2d_aligned.set_index("date")["residual"],
    "xgb_diff": xgb_diff_fy2d_aligned.set_index("date")["residual"],
    "lstm":     lstm_fy2d_aligned.set_index("date")["residual"],
    "lstm_ae":     lstm_ae_fy2d_aligned.set_index("date")["residual"],
}
res_fy2d, df_pr_fy2d = pr_curves_for_models_across_windows(
    model_series=model_series_fy2d,
    man_dates=man_dates_fy2d,
    window_days=(1,3,5),
    return_dataframe=True
)

In [27]:
model_series_saral = {
    "baseline": baseline_saral_aligned.set_index("date")["residual"],
    "arima":    arima_saral_aligned.set_index("date")["residual"],
    "xgb": xgb_saral_aligned.set_index("date")["residual"],
    "xgb_diff": xgb_diff_saral_aligned.set_index("date")["residual"],
    "lstm":     lstm_saral_aligned.set_index("date")["residual"],
    "lstm_ae":     lstm_ae_saral_aligned.set_index("date")["residual"],
}
res_saral, df_pr_saral = pr_curves_for_models_across_windows(
    model_series=model_series_saral,
    man_dates=man_dates_saral,
    window_days=(1,3,5),
    return_dataframe=True
)

In [28]:
# 
df_pr_s3a_ = df_pr_s3a.copy()
# df_pr_s3a_["sat"] = "S3A"

df_pr_fy2d_ = df_pr_fy2d.copy()
# df_pr_fy2d_["sat"] = "FY2D"

df_pr_saral_ = df_pr_saral.copy()
# df_pr_saral_["sat"] = "SARAL"

# 
df_pr_s3a_.to_csv("/project_final/result/pr_S3A.csv", index=False)
df_pr_fy2d_.to_csv("/project_final/result/pr_FY2D.csv", index=False)
df_pr_saral_.to_csv("/project_final/result/pr_SARAL.csv", index=False)

In [29]:
def _select_best_fbeta(group: pd.DataFrame, beta: float):

    g = group.copy()
    p = g["precision"].astype(float).to_numpy()
    r = g["recall"].astype(float).to_numpy()
    beta2 = beta * beta
    denom = beta2 * p + r
    fbeta = np.where(denom > 0, (1.0 + beta2) * p * r / denom, 0.0)
    g = g.assign(Fbeta=fbeta)

    g = g.sort_values(["Fbeta", "recall", "precision"], ascending=False)
    row = g.iloc[0]

    # 
    return pd.Series({
        "model":       row["model"],
        "window_days": int(row["window_days"]),
        "best_threshold": float(row["threshold"]),
        "best_precision": float(row["precision"]),
        "best_recall":    float(row["recall"]),

        f"best_F{int(beta) if float(beta).is_integer() else beta}": float(row["Fbeta"]),
    })

def select_best_fbeta(df_pr: pd.DataFrame, beta: float) -> pd.DataFrame:

    cols_needed = ["model","window_days","threshold","precision","recall"]
    df = df_pr[cols_needed].dropna(subset=["precision","recall"]).copy()
    best_df = (
        df.groupby(["model","window_days"], dropna=False)
          .apply(_select_best_fbeta, beta=beta)
          .reset_index(drop=True)
          .sort_values(["model","window_days"], ignore_index=True)
    )
    return best_df

def get_best_f1_f2(df_pr: pd.DataFrame):

    df_best_f1 = select_best_fbeta(df_pr, beta=1.0)
    df_best_f2 = select_best_fbeta(df_pr, beta=2.0)
    return df_best_f1, df_best_f2

In [30]:
bestF1_s3a, bestF2_s3a = get_best_f1_f2(df_pr_s3a)

In [31]:
def merge_best_f1_f2(df_pr: pd.DataFrame):
    f1 = select_best_fbeta(df_pr, 1.0).rename(columns={"best_threshold":"thr_F1","best_precision":"P_F1","best_recall":"R_F1","best_F1":"F1"})
    f2 = select_best_fbeta(df_pr, 2.0).rename(columns={"best_threshold":"thr_F2","best_precision":"P_F2","best_recall":"R_F2","best_F2":"F2"})
    merged = pd.merge(f1, f2, on=["model","window_days"], how="outer").sort_values(["model","window_days"], ignore_index=True)
    return merged

merged_s3a = merge_best_f1_f2(df_pr_s3a)

In [32]:
merged_s3a.to_csv("/project_final/result/metrics_s3a.csv", index=False)

In [33]:
bestF1_fy2d, bestF2_fy2d = get_best_f1_f2(df_pr_fy2d)

In [34]:
def merge_best_f1_f2(df_pr: pd.DataFrame):
    f1 = select_best_fbeta(df_pr, 1.0).rename(columns={"best_threshold":"thr_F1","best_precision":"P_F1","best_recall":"R_F1","best_F1":"F1"})
    f2 = select_best_fbeta(df_pr, 2.0).rename(columns={"best_threshold":"thr_F2","best_precision":"P_F2","best_recall":"R_F2","best_F2":"F2"})
    merged = pd.merge(f1, f2, on=["model","window_days"], how="outer").sort_values(["model","window_days"], ignore_index=True)
    return merged

merged_fy2d = merge_best_f1_f2(df_pr_fy2d)

In [35]:
merged_fy2d.to_csv("/project_final/result/metrics_fy2d.csv", index=False)

In [36]:
bestF1_saral, bestF2_saral = get_best_f1_f2(df_pr_saral)

In [37]:
def merge_best_f1_f2(df_pr: pd.DataFrame):
    f1 = select_best_fbeta(df_pr, 1.0).rename(columns={"best_threshold":"thr_F1","best_precision":"P_F1","best_recall":"R_F1","best_F1":"F1"})
    f2 = select_best_fbeta(df_pr, 2.0).rename(columns={"best_threshold":"thr_F2","best_precision":"P_F2","best_recall":"R_F2","best_F2":"F2"})
    merged = pd.merge(f1, f2, on=["model","window_days"], how="outer").sort_values(["model","window_days"], ignore_index=True)
    return merged

merged_saral = merge_best_f1_f2(df_pr_saral)

In [38]:
merged_s3a.sort_values(by=["window_days", "F2"], ascending=[True, False])

,model,window_days,thr_F1,P_F1,R_F1,F1,thr_F2,P_F2,R_F2,F2
3,baseline,1,7.426628e-08,0.942308,0.859649,0.899083,7.426628e-08,0.942308,0.859649,0.875000
0,arima,1,6.086100e-08,0.924528,0.859649,0.890909,6.086100e-08,0.924528,0.859649,0.871886
6,lstm,1,1.232125e-01,0.924528,0.859649,0.890909,1.232125e-01,0.924528,0.859649,0.871886
15,xgb_diff,1,1.809730e+00,0.812500,0.456140,0.584270,4.582364e-01,0.400000,0.771930,0.650888
9,lstm_ae,1,1.615874e-02,0.181818,0.807018,0.296774,1.532988e-02,0.177778,0.842105,0.481928
12,xgb,1,5.856881e-03,0.111111,0.333333,0.166667,5.856881e-03,0.111111,0.333333,0.238095
7,lstm,3,1.232125e-01,0.980769,0.894737,0.935780,1.232125e-01,0.980769,0.894737,0.910714
4,baseline,3,7.426628e-08,0.980392,0.877193,0.925926,4.376862e-08,0.866667,0.912281,0.902778
1,arima,3,6.086100e-08,0.961538,0.877193,0.917431,6.086100e-08,0.961538,0.877193,0.892857
16,xgb_diff,3,1.111794e+00,0.596774,0.649123,0.621849,3.010557e-01,0.404580,0.929825,0.738162


In [39]:
merged_saral.to_csv("/project_final/result/metrics_saral.csv", index=False)